In [1]:
import sys
from github import Github
import networkx as nx
from operator import itemgetter

# Global Variables
ACCESS_TOKEN = '05bb4eb867b152be20dd11f4fa292107c839931c'
USER = 'minrk'  # Define the GitHub User Name
REPO = 'findspark'  # Define the Repo name
client = Github(ACCESS_TOKEN)
g = nx.DiGraph()

In [2]:
def getStargazers(REPO):
    user = client.get_user(USER)
    repo = user.get_repo(REPO)  # Get a specific repo
    REPOS = user.get_repos()
    stargazers = list(repo.get_stargazers())  # The list of users who starred this REPO
    g.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)
    for sg in stargazers:
        g.add_node(sg.login + '(user)', type='user')
        g.add_edge(sg.login + '(user)', repo.name + '(repo)', type='gazes')
    #	print(len(stargazers))#See if it return a correct list
    return stargazers


def buildRelations(stargazers):
    for i, sg in enumerate(stargazers):
        try:
            for follower in sg.get_followers():
                if follower.login + '(user)' in g:
                    g.add_edge(follower.login + '(user)', sg.login + '(user)',
                               type='follows')
        except Exception:  # ssl.SSLError
            sys.stderr.write("Encountered an error fetching followers for", \
                             sg.login, "Skipping.")

        print("Processed", i + 1, " stargazers. Num nodes/edges in graph", \
              g.number_of_nodes(), "/", g.number_of_edges())
        print("Rate limit remaining", client.rate_limiting)

In [3]:
stargazers = getStargazers(REPO)
buildRelations(stargazers)
print(nx.info(g), '\n')
print(len([e for e in g.edges_iter(data=True) if e[2]['type'] == 'follows']), '\n')

Processed 1  stargazers. Num nodes/edges in graph 195 / 194
Rate limit remaining (4093, 5000)
Processed 2  stargazers. Num nodes/edges in graph 195 / 198
Rate limit remaining (4069, 5000)
Processed 3  stargazers. Num nodes/edges in graph 195 / 201
Rate limit remaining (4066, 5000)
Processed 4  stargazers. Num nodes/edges in graph 195 / 202
Rate limit remaining (4064, 5000)
Processed 5  stargazers. Num nodes/edges in graph 195 / 204
Rate limit remaining (4061, 5000)
Processed 6  stargazers. Num nodes/edges in graph 195 / 209
Rate limit remaining (4043, 5000)
Processed 7  stargazers. Num nodes/edges in graph 195 / 210
Rate limit remaining (4041, 5000)
Processed 8  stargazers. Num nodes/edges in graph 195 / 210
Rate limit remaining (4040, 5000)
Processed 9  stargazers. Num nodes/edges in graph 195 / 216
Rate limit remaining (4031, 5000)
Processed 10  stargazers. Num nodes/edges in graph 195 / 216
Rate limit remaining (4028, 5000)
Processed 11  stargazers. Num nodes/edges in graph 195 / 21

Processed 88  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3860, 5000)
Processed 89  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3859, 5000)
Processed 90  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3856, 5000)
Processed 91  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3855, 5000)
Processed 92  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3854, 5000)
Processed 93  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3851, 5000)
Processed 94  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3850, 5000)
Processed 95  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3849, 5000)
Processed 96  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3848, 5000)
Processed 97  stargazers. Num nodes/edges in graph 195 / 243
Rate limit remaining (3847, 5000)
Processed 98  stargazers. Num nodes/edges in graph

Processed 174  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3730, 5000)
Processed 175  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3729, 5000)
Processed 176  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3728, 5000)
Processed 177  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3727, 5000)
Processed 178  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3713, 5000)
Processed 179  stargazers. Num nodes/edges in graph 195 / 254
Rate limit remaining (3712, 5000)
Processed 180  stargazers. Num nodes/edges in graph 195 / 255
Rate limit remaining (3703, 5000)
Processed 181  stargazers. Num nodes/edges in graph 195 / 255
Rate limit remaining (3702, 5000)
Processed 182  stargazers. Num nodes/edges in graph 195 / 255
Rate limit remaining (3701, 5000)
Processed 183  stargazers. Num nodes/edges in graph 195 / 255
Rate limit remaining (3700, 5000)
Processed 184  stargazers. Num nodes/edg

In [4]:
def getHottestRepo(stargazers):
    MAX_REPOS = 5
    for i, sg in enumerate(stargazers):
        print(sg.login)
        try:
            for starred in sg.get_starred()[:MAX_REPOS]:  # Slice to avoid supernodes
                g.add_node(starred.name + '(repo)', type='repo', lang=starred.language, \
                           owner=starred.owner.login)
                g.add_edge(sg.login + '(user)', starred.name + '(repo)', type='gazes')
        except Exception:  # ssl.SSLError:
            print("Encountered an error fetching starred repos for", sg.login, "Skipping.")

        print("Processed", i + 1, "stargazers' starred repos")
        print("Num nodes/edges in graph", g.number_of_nodes(), "/", g.number_of_edges())
        print("Rate limit", client.rate_limiting)
    print(nx.info(g), '\n')
    # repos = [n for n in g.nodes_iter() if g.node[n]['type'] == 'repo']
    print("Popular repositories")
#     for (n,d) in g.in_degree_iter():
#         node = g.node[n]
#         if node['type'] == 'repo':
#             print(1)
    print(sorted([(n, d)
                  for (n, d) in g.in_degree_iter()
                  if g.node[n]['type'] == 'repo'], key=itemgetter(1), reverse=True)[:10])

    print("Respositories that luzhijun has bookmarked")
    print([(n, g.node[n]['lang'])
           for n in g['luzhijun(user)']
           if g['luzhijun(user)'][n]['type'] == 'gazes'])


    print("Programming languages luzhijun is interested in")
    print(list(set([g.node[n]['lang']
                    for n in g['luzhijun(user)']
                    if g['luzhijun(user)'][n]['type'] == 'gazes'])))

    print("Supernode candidates")
    print(sorted([(n, len(g.out_edges(n)))
                  for n in g.nodes_iter()
                  if g.node[n]['type'] == 'user' and len(g.out_edges(n)) > MAX_REPOS], \
                 key=itemgetter(1), reverse=True))

In [5]:
getHottestRepo(stargazers)

pchalasani
Processed 1 stargazers' starred repos
Num nodes/edges in graph 200 / 260
Rate limit (3688, 5000)
rgbkrk
Processed 2 stargazers' starred repos
Num nodes/edges in graph 205 / 265
Rate limit (3687, 5000)
esafak
Processed 3 stargazers' starred repos
Num nodes/edges in graph 210 / 270
Rate limit (3686, 5000)
rdhyee
Processed 4 stargazers' starred repos
Num nodes/edges in graph 215 / 275
Rate limit (3685, 5000)
rholder
Processed 5 stargazers' starred repos
Num nodes/edges in graph 220 / 280
Rate limit (3684, 5000)
freeman-lab
Processed 6 stargazers' starred repos
Num nodes/edges in graph 225 / 285
Rate limit (3683, 5000)
he0x
Processed 7 stargazers' starred repos
Num nodes/edges in graph 230 / 290
Rate limit (3682, 5000)
cbouey
Processed 8 stargazers' starred repos
Num nodes/edges in graph 235 / 295
Rate limit (3681, 5000)
stared
Processed 9 stargazers' starred repos
Num nodes/edges in graph 240 / 300
Rate limit (3680, 5000)
ryan-williams
Processed 10 stargazers' starred repos
Num

Processed 78 stargazers' starred repos
Num nodes/edges in graph 572 / 643
Rate limit (3611, 5000)
beflew
Processed 79 stargazers' starred repos
Num nodes/edges in graph 573 / 644
Rate limit (3610, 5000)
jackmaney
Processed 80 stargazers' starred repos
Num nodes/edges in graph 577 / 649
Rate limit (3609, 5000)
analyticalmonk
Processed 81 stargazers' starred repos
Num nodes/edges in graph 582 / 654
Rate limit (3608, 5000)
telamonian
Processed 82 stargazers' starred repos
Num nodes/edges in graph 587 / 659
Rate limit (3607, 5000)
PaulLiang1
Processed 83 stargazers' starred repos
Num nodes/edges in graph 592 / 664
Rate limit (3606, 5000)
jmgpeeters
Processed 84 stargazers' starred repos
Num nodes/edges in graph 597 / 669
Rate limit (3605, 5000)
LiutongZhou
Processed 85 stargazers' starred repos
Num nodes/edges in graph 602 / 674
Rate limit (3604, 5000)
gomaster-me
Processed 86 stargazers' starred repos
Num nodes/edges in graph 607 / 679
Rate limit (3603, 5000)
suresh2692
Processed 87 starg

Processed 154 stargazers' starred repos
Num nodes/edges in graph 901 / 1007
Rate limit (3535, 5000)
alisonzhu
Processed 155 stargazers' starred repos
Num nodes/edges in graph 905 / 1011
Rate limit (3534, 5000)
bossjones
Processed 156 stargazers' starred repos
Num nodes/edges in graph 909 / 1016
Rate limit (3533, 5000)
YachaoZhu
Processed 157 stargazers' starred repos
Num nodes/edges in graph 914 / 1021
Rate limit (3532, 5000)
charrtay
Processed 158 stargazers' starred repos
Num nodes/edges in graph 917 / 1026
Rate limit (3531, 5000)
7125messi
Processed 159 stargazers' starred repos
Num nodes/edges in graph 919 / 1031
Rate limit (3530, 5000)
dufferzafar
Processed 160 stargazers' starred repos
Num nodes/edges in graph 923 / 1036
Rate limit (3529, 5000)
andfanilo
Processed 161 stargazers' starred repos
Num nodes/edges in graph 926 / 1041
Rate limit (3528, 5000)
asmith26
Processed 162 stargazers' starred repos
Num nodes/edges in graph 930 / 1046
Rate limit (3527, 5000)
Gmousse
Processed 16